In [ ]:
import platform,cpuinfo
platform.processor()
cpuinfo.get_cpu_info()

In [2]:
!apt-get install sox ffmpeg libcairo2 libcairo2-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libcairo2 is already the newest version (1.16.0-5ubuntu2).
libcairo2 set to manually installed.
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  libblkid-dev libcairo-script-interpreter2 libffi-dev libglib2.0-dev
  libglib2.0-dev-bin libice-dev liblzo2-2 libmount-dev libopencore-amrnb0
  libopencore-amrwb0 libpixman-1-dev libselinux1-dev libsepol-dev libsm-dev
  libsox-fmt-alsa libsox-fmt-base libsox3 libwavpack1 libxcb-render0-dev
  libxcb-shm0-dev
Suggested packages:
  libcairo2-doc libgirepository1.0-dev libglib2.0-doc libgdk-pixbuf2.0-bin
  | libgdk-pixbuf2.0-dev libxml2-utils libice-doc libsm-doc libsox-fmt-all
The following NEW packages will be installed:
  libblkid-dev libcairo-script-interpreter2 libcairo2-dev libffi-dev
  libglib2.0-dev libglib2.0-dev-bin libice-dev liblzo2-2 libmount-dev
  libopencore-amrnb0 li

In [3]:
!pip install tensorflow-hub tensorflow-text pipdeptree
!pip install wheel jedi>=0.16 typing-extensions>=4.6.1,!=4.7.0 pycairo>=1.16.0
!pip install -U tensorflow
!pip install -U tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 18.3 MB/s eta 0:00:00


In [4]:
!pipdeptree --reverse --packages typing-extensions

Warning!! Cyclic dependencies found:
* sphinxcontrib-applehelp => Sphinx => sphinxcontrib-applehelp
* sphinxcontrib-devhelp => Sphinx => sphinxcontrib-devhelp
* sphinxcontrib-htmlhelp => Sphinx => sphinxcontrib-htmlhelp
* sphinxcontrib-qthelp => Sphinx => sphinxcontrib-qthelp
* sphinxcontrib-serializinghtml => Sphinx => sphinxcontrib-serializinghtml
* Sphinx => sphinxcontrib-applehelp => Sphinx
* triton => torch => triton
* torch => triton => torch
------------------------------------------------------------------------
typing-extensions==4.5.0
├── arviz==0.15.1 [requires: typing-extensions>=4.1.0]
│   └── pymc==5.7.2 [requires: arviz>=0.13.0]
├── chex==0.1.7 [requires: typing-extensions>=4.2.0]
│   └── optax==0.1.7 [requires: chex>=0.1.5]
│       └── flax==0.7.2 [requires: optax]
│           └── dopamine-rl==4.0.6 [requires: flax>=0.2.0]
├── flax==0.7.2 [requires: typing-extensions>=4.1.1]
│   └── dopamine-rl==4.0.6 [requires: flax>=0.2.0]
├── inflect==7.0.0 [requires: typing-extensio

In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
from scipy.spatial.distance import cosine

# Load CoNLL-U data
def load_conllu_data(file_path):
    sentences = []
    sentence = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.startswith('#'):
                continue
            if not line.strip():
                sentences.append(sentence)
                sentence = []
                continue
            fields = line.strip().split('\t')
            word = fields[1]
            sentence.append(word)
    return sentences

# Load data
sentences_esl = load_conllu_data('/content/drive/MyDrive/Train_dat/en_esl-ud-train.conllu')
sentences_eslspok = load_conllu_data('/content/drive/MyDrive/Train_dat/en_eslspok-ud-train.conllu')
sentences_en_gum = load_conllu_data('/content/drive/MyDrive/Train_dat/en_gum-ud-train.conllu')
sentences_en_partut = load_conllu_data('/content/drive/MyDrive/Train_dat/en_partut-ud-train.conllu')
sentences_en_ewt = load_conllu_data('/content/drive/MyDrive/Train_dat/en_ewt-ud-train.conllu')
sentences_fa_seraj = load_conllu_data('/content/drive/MyDrive/Train_dat/fa_seraji-ud-train.conllu')
sentences_fa_perdt = load_conllu_data('/content/drive/MyDrive/Train_dat/fa_perdt-ud-train.conllu')

# Load pre-trained multilingual BERT model
bert_model = hub.load("https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4")
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3")

# Encode sentences
def encode_sentences(sentences):
    sentence_embeddings = []
    for sentence in sentences:
        sentence_str = ' '.join(sentence)
        input_data = preprocessor(tf.constant([sentence_str]))
        outputs = bert_model(input_data)
        sentence_embedding = outputs['pooled_output'][0].numpy()
        sentence_embeddings.append(sentence_embedding)
    return sentence_embeddings

sentence_embeddings_esl = encode_sentences(sentences_esl)
sentence_embeddings_eslspok = encode_sentences(sentences_eslspok)
sentence_embeddings_en_gum = encode_sentences(sentences_en_gum)
sentence_embeddings_en_partut = encode_sentences(sentences_en_partut)
sentence_embeddings_en_ewt = encode_sentences(sentences_en_ewt)
sentence_embeddings_fa_seraj = encode_sentences(sentences_fa_seraj)
sentence_embeddings_fa_perdt = encode_sentences(sentences_fa_perdt)
#sentence_embeddings1 = encode_sentences(sentences1)
#sentence_embeddings2 = encode_sentences(sentences2)

# Compare sentence embeddings
#for i, embedding1 in enumerate(sentence_embeddings1):
#    for j, embedding2 in enumerate(sentence_embeddings2):
#        similarity = 1 - cosine(embedding1, embedding2)
#        print(f'Similarity between sentence {i+1} in file 1 and sentence {j+1} in file 2: {similarity:.2f}')


KeyboardInterrupt: ignored

In [ ]:
# Compare sentence embeddings and append results to a text file(example)
with open('en_esl_fa_perdt.txt', 'a') as f:
    first_line="first_line_no,second_line_no,Similarity"
    f.write(first_line)
    for i, embedding1 in enumerate(sentence_embeddings1):
        for j, embedding2 in enumerate(sentence_embeddings2):
            similarity = 1 - cosine(embedding1, embedding2)
            #result = f'Similarity between, sentence {i+1} in file 1 and sentence {j+1} in file 2: {similarity:.2f}\n'
            result ="line{},line{},{:.4f}".format(i+1,j+1,similarity)
            #print(result)
            f.write(result)


In [6]:
# Define a function to compare and write similarities to a file
def compare_and_write_similarities(file1_name, sentences1, file2_name, sentences2, output_file):
    with open(output_file, 'w') as f:
        #f.write(f"Comparison between {file1_name} and {file2_name}:\n")
        #f.write("Sentence1, Sentence2, Similarity\n")
        first_line="first_line_no,second_line_no,Similarity"
        f.write(first_line)
        for i, embedding1 in enumerate(sentences1):
            for j, embedding2 in enumerate(sentences2):
                similarity = 1 - cosine(embedding1, embedding2)
                result = "line{},line{},{:.4f}".format(i+1,j+1,similarity)
                f.write(result)

In [ ]:
datasets1 = [
    ("en_esl", sentence_embeddings_esl),
    ("en_eslspok", sentence_embeddings_eslspok),
    ("en_gum", sentence_embeddings_en_gum),
    ("en_partut", sentence_embeddings_en_partut),
    ("en_ewt", sentence_embeddings_en_ewt)
]
datasets2 = [
    ("fa_seraj", sentence_embeddings_fa_seraj),
    ("fa_perdt", sentence_embeddings_fa_perdt),
]
for file1_name,sentences1 in datasets1:
  for file2_name,sentences2 in datasets2:
    output_file = f"{file1_name}_{file2_name}_comparison.txt"
    compare_and_write_similarities(file1_name, sentences1, file2_name, sentences2, output_file)

In [ ]:
import threading
import time
import psutil

class CompareAndWriteSimilaritiesThread(threading.Thread):
    def __init__(self, file1_name, sentences1, file2_name, sentences2, output_file, lock):
        super().__init__()
        self.file1_name = file1_name
        self.sentences1 = sentences1
        self.file2_name = file2_name
        self.sentences2 = sentences2
        self.output_file = output_file
        self.lock = lock

    def run(self):
        with open(self.output_file, 'a') as f:
            for i, embedding1 in enumerate(self.sentences1):
                for j, embedding2 in enumerate(self.sentences2):
                    similarity = 1 - cosine(embedding1, embedding2)
                    result = "line{},line{},{:.4f}".format(i+1,j+1,similarity)
                    self.lock.acquire()
                    f.write(result)
                    self.lock.release()

def compare_and_write_similarities_parallel(file1_name, sentences1, file2_name, sentences2, output_file):
    lock = threading.Lock()
    threads = []
    for i, embedding1 in enumerate(sentences1):
        for j, embedding2 in enumerate(sentences2):
            thread = CompareAndWriteSimilaritiesThread(file1_name, sentences1, file2_name, sentences2, output_file, lock)
            threads.append(thread)
            thread.start()

    for thread in threads:
        thread.join()

#operation 33
datasets1 = [
    ("en_esl", sentence_embeddings_esl),
    ("en_eslspok", sentence_embeddings_eslspok),
    ("en_gum", sentence_embeddings_en_gum),
    ("en_partut", sentence_embeddings_en_partut),
    ("en_ewt", sentence_embeddings_en_ewt)
]
datasets2 = [
    ("fa_seraj", sentence_embeddings_fa_seraj),
    ("fa_perdt", sentence_embeddings_fa_perdt),
]

# Take system memory into account
available_memory = psutil.virtual_memory().available
# Limit the number of threads to avoid overloading the system
max_threads = int(available_memory / (1024 * 1024))
if max_threads < 1:
    max_threads = 1

for file1_name,sentences1 in datasets1:
  for file2_name,sentences2 in datasets2:
    output_file = f"{file1_name}_{file2_name}_comparison.txt"
    compare_and_write_similarities_parallel(file1_name, sentences1, file2_name, sentences2, output_file)

In [ ]:
!tar -cf archive.tar.gz *.txt -I "gzip --best"
!mv archive.tar.gz /content/drive/MyDrive/Export
